In [ ]:
!pip install --quiet fastapi nest_asyncio uvicorn pyngrok pydantic
!pip install --quiet diffusers transformers accelerate peft safetensors huggingface_hub google-generativeai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.4 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import io, base64, re

import torch
from diffusers import StableDiffusionPipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline as hf_pipeline
from peft import PeftModel


In [ ]:
import google.generativeai as genai
API_KEY = "YOUR_API_KEY"
genai.configure(api_key=API_KEY)
def build_instruction(story, keyword, gender):
    gender = gender.lower().strip()
    if gender == "girl":
        character_phrase = "<wucs> little girl with black hair wearing a pink shirt"
    elif gender == "boy":
        character_phrase = "<lbcs> little boy with brown hair wearing a light blue shirt"
    else:
        raise ValueError("Gender must be 'girl' or 'boy'.")

    instruction = (
        f"You are helping me generate prompts for Stable Diffusion 1.5 using DreamBooth and Textual Inversion.\n\n"
        f"Note: The tokens <wucs> and <lbcs> are DreamBooth-trained character tokens used to preserve character identity in images.\n"
        f"The token <stylii> is a Textual Inversion token trained for a consistent Pixar-style visual.\n\n"
        f"*Your goal is to write short, clean prompts that result in consistent Pixar-style images with the correct character pose, facial expression, and a clearly grounded object.*\n\n"
        f"🔑 Prompt Format Rules:\n"
        f"1. Start each prompt with: *A Pixar-style portrait of a single {character_phrase}*\n"
        f"2. Include a consistent background setting relevant to the story, such as a school, a classroom, a library, or a yard. Always state the setting explicitly in each prompt.\n"
        f"3. Focus ONLY on the main character — do not include any other people or animals.\n"
        f"4. Each prompt must include the keyword: *{keyword}*, clearly described and visually grounded (e.g., 'a shoe on the ground').\n"
        f"5. No interaction between the character and the object — describe where the object is (e.g., 'a shoe placed near her right side').\n"
        f"6. Maintain this order for every prompt:\n"
        f"   - Character appearance\n"
        f"   - Facial expression (e.g., 'smiling softly', 'looking curious')\n"
        f"   - Pose (e.g., 'standing', 'sitting')\n"
        f"   - Clearly describe the object’s position (e.g., 'a shoe is on the floor next to her')\n"
        f"   - Setting (e.g., 'in a library')\n"
        f"   - End with: *in the style of <stylii>*\n"
        f"7. Use physically realistic, concrete descriptions. Do not use metaphors or abstract phrases.\n\n"
        f"Now, break the story below into *3 distinct visual scenes*, and write 1 prompt per scene following the above rules.\n\n"
        f"⚠ Respond ONLY with a plain Python list of strings (no variable name, no markdown):\n"
        f"[\n"
        f"    \"...\",\n"
        f"    \"...\",\n"
        f"    \"...\"\n"
        f"]\n\n"
        f"Story:\n\"\"\"\n{story}\n\"\"\"\n"
    )
    return instruction


In [ ]:
def translate_story_to_arabic(story, model):
    """
    Translates a short English children's story into Modern Standard Arabic (Fusha),
    keeping it expressive and fun for preschoolers. Preserves scene-by-scene format.

    Args:
        story_english (str): The English story input.
        model: The model object used for generating content (e.g., from OpenAI or similar).

    Returns:
        str: The translated story in Arabic.
    """
    prompt = f"""
    Please translate the following short children’s story into Modern Standard Arabic (Fusha) in a way that is emotionally expressive, fun, and engaging for preschoolers (ages 3–5).

    You are encouraged to be creative — do not translate the story literally. Feel free to adapt or expand it slightly to better suit Arabic culture and language, while preserving the warm tone and original meaning.

    Only output the translated story in Arabic. Do not include any explanations or extra commentary. Keep the same scene-by-scene structure and formatting as the input.

    Here is the story:

    {story}
    """
    response = model.generate_content(prompt)
    return response.text

In [ ]:
def split_arabic_scenes(arabic_translation: str):
    # 1. Remove all newline characters
    clean_text = arabic_translation.replace("\n", "").strip()

    # 2. Split by "المشهد" keyword
    parts = clean_text.split("المشهد")

    # 3. Reattach "المشهد" and remove empty results
    arabic_scenes = [
        f"المشهد{part.strip()}" for part in parts if part.strip()
    ]

    return arabic_scenes

In [ ]:
# Step 3: LLaMA2 + LoRA story generator setup
# ─────────────────────────────────────────────
hf_token = "YOUR_HF_TOKEN"
base_model_id = "meta-llama/Llama-2-7b-hf"
adapter_repo   = "layanB/Llama2-7B-LoRA-Adapter"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True, use_auth_token=hf_token)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=hf_token
)
story_model = PeftModel.from_pretrained(base_model, adapter_repo, use_auth_token=hf_token)
story_gen = hf_pipeline("text-generation", model=story_model, tokenizer=tokenizer)

def split_into_scenes(text):
    scenes = re.split(r'Scene \d+:', text)
    if scenes and not scenes[0].strip():
        scenes = scenes[1:]
    return [s.strip() for s in scenes]


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:934: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# ─────────────────────────────────────────────
# Step 4: DreamBooth pipelines setup
# ─────────────────────────────────────────────
def make_pipe(repo_id: str):
    pipe = StableDiffusionPipeline.from_pretrained(
        repo_id,
        torch_dtype=torch.float16,
        safety_checker=None,
        low_cpu_mem_usage=True
    ).to("cuda")
    pipe.enable_attention_slicing()
    return pipe

girl_pipe = make_pipe("LeenSMN/deployingWindUp_200Steps")
boy_pipe  = make_pipe("LeenSMN/deployingLuca_400Steps")

for pipe, repo in [(girl_pipe,"LeenSMN/deployingWindUp_200Steps"),
                   (boy_pipe,"LeenSMN/deployingLuca_400Steps")]:
    tok = AutoTokenizer.from_pretrained(repo, subfolder="tokenizer")
    pipe.tokenizer = tok
    pipe.text_encoder.resize_token_embeddings(len(tok))

for pipe in (girl_pipe, boy_pipe):
    pipe.load_textual_inversion(
        pretrained_model_name_or_path="LeenSMN/deployingEmbeddings",
        weight_name="learned_embeds-steps-2000.safetensors",
        token="<stylii>"
    )

def encode_image(img):
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return base64.b64encode(buf.getvalue()).decode("utf-8")

model_index.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


learned_embeds-steps-2000.safetensors:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
# Define your request model
class GenerationRequest(BaseModel):
    name: str
    word: str
    gender: str  # "girl" or "boy"

# Initialize FastAPI app
app = FastAPI()


from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Or replace with specific origin: ["http://127.0.0.1:5500"]
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/generate")
async def generate(req: GenerationRequest):
    # 1 Generate story scenes
    prompt = f"""[INST] Write a short story in 3 simple scenes for kids aged 3–5.
    Use a {req.gender} main character named '{req.name}' and the keyword '{req.word}'.
    Make sure the story helps kids learn and remember the word '{req.word}'.[/INST]"""

    out = story_gen(prompt, max_new_tokens=120)[0]["generated_text"].replace(prompt, "").strip()
    scenes = split_into_scenes(out)

    # 2️ Build Gemini instruction & call
    instruction = build_instruction("\n\n".join(scenes), req.word, req.gender)
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(instruction)
    prompts = eval(response.text.strip())

    # Translate story to arabic
    arabic_translation = translate_story_to_arabic(out, model)
    arabic_scenes = split_arabic_scenes(arabic_translation)

    # 3️ Generate images
    pipe = girl_pipe if req.gender.lower() == "girl" else boy_pipe
    images = []
    for p in prompts:
        img = pipe(p, num_inference_steps=60, guidance_scale=9.0, height=512, width=512).images[0]
        images.append(encode_image(img))

    return {"scenes": scenes, "images": images, "arabic scenes": arabic_scenes}

# ─────────────────────────────────────────────
# Step 6: Expose via ngrok & run Uvicorn
# ─────────────────────────────────────────────
ngrok.set_auth_token("ngrok_auth_token")
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}/generate")

uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://d8f562316923.ngrok-free.app" -> "http://localhost:8000"/generate


INFO:     Started server process [1517]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     176.28.141.236:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     37.202.78.248:0 - "OPTIONS /generate HTTP/1.1" 200 OK


  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

...

INFO:     37.202.78.248:0 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

INFO:     37.202.78.248:0 - "POST /generate HTTP/1.1" 200 OK


In [ ]:
#from pyngrok import ngrok
#ngrok.kill()  # ✅ This kills any previous tunnel
